# Installation and Setup

In [ ]:
## for data
import json
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
import os
import shutil


# Read Data



In [ ]:
## Google Colab- Google Drive connections instructions 

# This step needs to be done for the first time when you're reading something from the Shared Project Folder 
# Please navigate to "Shared with me" on your Gdrive home sceen
# Right click on the "NLP-Group1-FinalProj" (the primary folder for this project) and select "Add shortcut to Drive"
# This way the below code can find a link to the Project drive folder through your own drive.


# Mounting your personal Gdrive to the Colab notebook
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
groc_data = pd.read_csv('/content/drive/My Drive/NLP-Group1-FinalProj/Data/Raw/groc_amz_rev.csv.gz', sep=',', compression="gzip")

# Data Preparation

In [ ]:
groc_data = groc_data.head(10000)
star_revs = groc_data.loc[groc_data.rating==1]
groc_data = groc_data['review_text'].squeeze()

# Laod Model and Prediction